In [1]:
from lxml import etree  # pip install lxml
import numpy as np
import cv2
import glob

In [2]:
def polygon_as_2darray(polygon_str):
    '''
    Conversts a polygon in the string format required by CVAT into the 2d numpy array format required by cv2.polylines. 
    '''
    points_str = polygon_str.replace(';', ',') 
    points_flt = [float(i) for i in points_str.split(',')]
    points_int = [int(i) for i in points_flt]           
    points = np.array(points_int, np.int32)
    points_2d = np.reshape(points, (-1, 2))
    return points_2d
    
# polygon_str =  "25.00,280.50;12.00,280.50;4.50,274.00;65.50,219.00;65.50,240.00;50.50,264.00;25.00,280.50"  
# polygon_as_2darray(polygon_str)

In [3]:
# MAIN

XML_PATH = '../output/detected_objects.xml'
IMAGE_DIR = '../rawdata'
ANNOTATED_IMAGE_DIR = '../annotated'

# Get a list of image files sorted alphabetically. 
# This list is used as an index for the images.
# image_files[n-1] will return the path for the nth image.
# This is not good practice, the image file name should be stored in the XML file instead.

image_files = sorted(glob.glob(f'{IMAGE_DIR}/*.jpg'))

root = etree.parse(XML_PATH)
# print(etree.tostring(root, pretty_print=True).decode())

# Find all images which have polygons

images_with_polygon = root.xpath('//image/polygon/parent::image')

for image_with_polygon in images_with_polygon:
    id = image_with_polygon.get('id')
    print(id)
    
    # Get image 
    
    image_path = image_files[int(id)-1]
    img = cv2.imread(image_path)

    # Draw rectangles
    # <box label="light" occluded="0" xtl="8.00" ytl="186.00" xbr="208.00" ybr="1223.00"></box>
    
    for box in image_with_polygon.xpath(f'//image[@id={id}]/box'):
        xtl = box.get('xtl')
        ytl = box.get('ytl')
        xbr = box.get('xbr')
        ybr = box.get('ybr')
        start_point = ( int(round(float(xtl))), int(round(float(ytl))))
        end_point = ( int(round(float(xbr))), int(round(float(ybr))))
        print(start_point, end_point)
        img = cv2.rectangle(img, pt1=start_point, pt2=end_point, color=(0, 0, 255), thickness=2)
    
    # Draw polygons
    
    for polygon_points in image_with_polygon.xpath(f'//image[@id={id}]/polygon/@points'):
        polygon_2d = polygon_as_2darray(polygon_points)
        print(polygon_2d)
        img = cv2.polylines(img, pts=[polygon_2d], isClosed=False, color=(0, 255, 255), thickness=2)
            
    # Save image
    
    annotated_image_path = image_path.replace('rawdata', 'annotated')
    cv2.imwrite(annotated_image_path, img)    
    
    print('---')

155
(230, 0) (747, 796)
(653, 199) (1419, 1047)
(15, 105) (370, 688)
(1321, 791) (1602, 1063)
[[ 25 280]
 [ 12 280]
 [  4 274]
 [  2 251]
 [ 14 224]
 [ 27 207]
 [ 52 208]
 [ 65 219]
 [ 65 240]
 [ 50 264]
 [ 25 280]]
[[451  57]
 [426  49]
 [419  43]
 [415  20]
 [421   4]
 [451   3]
 [462  12]
 [474   6]
 [481   7]
 [484  35]
 [464  41]
 [456  55]
 [451  57]]
---
218
(1230, 356) (1839, 1033)
(579, 56) (1213, 977)
(0, 13) (289, 664)
[[ 985  177]
 [ 963  172]
 [ 951  160]
 [ 956  128]
 [ 964  105]
 [ 980   83]
 [ 999   66]
 [1010   65]
 [1017   76]
 [1014  104]
 [1001  134]
 [ 995  164]
 [ 985  177]]
---
227
(352, 0) (1651, 621)
[[1542  571]
 [1473  564]
 [1407  549]
 [1374  535]
 [1371  527]
 [1377  518]
 [1377  509]
 [1394  499]
 [1398  473]
 [1422  431]
 [1423  420]
 [1412  410]
 [1411  402]
 [1403  393]
 [1405  375]
 [1412  369]
 [1432  364]
 [1514  366]
 [1538  370]
 [1619  418]
 [1660  455]
 [1669  499]
 [1669  519]
 [1657  546]
 [1642  558]
 [1617  565]
 [1542  571]]
---
231
(11, 0)